In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sys
import pandas_redshift as pr
import json
import requests

In [2]:
job_start=dt.datetime.utcnow()

#### Intake Data

In [3]:
folder="/Users/peter.gurka/Google Drive/My Drive/Self Serve Reporting/API 2 API Project Data Request/"
input_df=pd.read_excel(folder + "Intake Sheet - API2.xlsx", sheet_name='Data', usecols='A')
request_df=pd.read_excel(folder + "Intake Sheet - API2.xlsx", sheet_name='Request', usecols='A:B', dtype={'B':str})

In [4]:
request_df

,Field,Value
0,Initials of Requester,OR
1,Request Title,OliviaThinkNowDrops
2,Provider,Think Now'
3,Activity ID,NaN
4,Start Date,2022-02-03 00:00:00
5,End Date,2022-02-03 00:00:00
6,Conversion Status ID,NaN
7,Conversion Status ID - NULL,Yes
8,User IDs,NaN
9,Business Line,NaN


In [5]:
input_df.head()

,Attributes
0,provider
1,activity_id
2,user_id
3,datetime_pt
4,survey_status_name


In [6]:
date = dt.datetime.today().date()

initials = request_df['Value'][0]
subject = request_df['Value'][1]

In [7]:
#WHERE FILTERS
provider = request_df['Value'][2]
activity_id = request_df['Value'][3]
start_date = request_df['Value'][4]
end_date = request_df['Value'][5]
conversion_status = request_df['Value'][6]
conversion_status_null = request_df['Value'][7]
user_id = request_df['Value'][8]
business_line = request_df['Value'][9]

In [8]:
from random import randint
rand=randint(1, 99)

#### Post on Slack

In [9]:
webhook_url = 'https://xxxx'

In [10]:
slack_message_start = """{0}'s API2 (CSM) request for {1} has started""".format(initials, subject)
slack_data = {'text': slack_message_start}

In [11]:
response = requests.post(
    webhook_url, data = json.dumps(slack_data),
    headers={'Content-Type': 'application/json'})
if response.status_code != 200:
    raise ValueError(
        'Request to slack returned an error %s, the response is:\n%s'
        % (response.status_code, response.text))

#### Query Elements

In [12]:
#WHERE FILTERS
filters = []

if pd.notnull(provider):
    filters.append("""provider in ('{provider})""".format(provider=provider))
if pd.notnull(activity_id):
    filters.append("""vuab.activity_id in ('{activity_id})""".format(activity_id=activity_id))
if pd.notnull(start_date):
    filters.append("""DATE(vuab.click_event_datetime_pt) >= '{start_date}'""".format(start_date=start_date))
if pd.notnull(end_date):
    filters.append("""DATE(vuab.click_event_datetime_pt) <= '{end_date}'""".format(end_date=end_date))
if pd.notnull(user_id):
    filters.append("""vuab.user_id in ({user_id})""".format(user_id=user_id))
if pd.notnull(business_line):
    filters.append("""vuab.business_line = '{business_line}'""".format(business_line=business_line))

#Conversion Filters
if pd.notnull(conversion_status) and conversion_status_null=='Yes':
    filters.append("""(vuab.conversion_status_id in ({conversion_status}) or (vuab.conversion_status_id IS NULL))"""
                   .format(conversion_status=conversion_status))
elif pd.notnull(conversion_status) and conversion_status_null=='No':
    filters.append("""vuab.conversion_status_id in ({conversion_status})"""
                   .format(conversion_status=conversion_status))
elif pd.isnull(conversion_status) and conversion_status_null=='Yes':
    filters.append("""vuab.conversion_status_id IS NULL""")

In [13]:
#ATTRIBUTES
attributes_list = list(input_df['Attributes'])
attributes = []

if 'provider' in attributes_list:
    attributes.append("""CASE WHEN vuab.provider_id != 18 THEN vuab.provider_name WHEN vuab.provider_id = 18 THEN p.provider_buyer_name END AS provider""")
if 'activity_id' in attributes_list:
    attributes.append("""vuab.activity_id AS survey_id""")
if 'user_id' in attributes_list:
    attributes.append("""vuab.user_id""")
if 'datetime_pt' in attributes_list:
    attributes.append("""vuab.click_event_datetime_pt""")
if 'survey_status_name' in attributes_list:
    attributes.append("""CASE WHEN vuab.conversion_status_id IS NULL THEN 'DropOff' ELSE CONCAT(CONCAT(ss.survey_status_id, ' '), ss.survey_status_name) END AS survey_status_name""")
if 'click_financial_cpi' in attributes_list:
    attributes.append("""vuab.click_financial_cpi""")
if 'financial_points_promised' in attributes_list:
    attributes.append("""vuab.financial_points_promised""")
if 'click_uuid' in attributes_list:
    attributes.append("""vuab.click_uuid""")

#### Query Strings

In [14]:
filters_str = ' and '.join(filters)
filters_str

"provider in ('Think Now') and DATE(vuab.click_event_datetime_pt) >= '2022-02-03 00:00:00' and DATE(vuab.click_event_datetime_pt) <= '2022-02-03 00:00:00' and vuab.conversion_status_id IS NULL"

In [15]:
attributes_str = ', '.join(attributes)
attributes_str

"CASE WHEN vuab.provider_id != 18 THEN vuab.provider_name WHEN vuab.provider_id = 18 THEN p.provider_buyer_name END AS provider, vuab.activity_id AS survey_id, vuab.user_id, vuab.click_event_datetime_pt, CASE WHEN vuab.conversion_status_id IS NULL THEN 'DropOff' ELSE CONCAT(CONCAT(ss.survey_status_id, ' '), ss.survey_status_name) END AS survey_status_name"

#### Data Request Pull

In [16]:
# tablename = """stg_data_genie.API2_{0}_{1}_{2}_{3}""".format(date.strftime("%Y-%m-%d").replace("-",""), initials, subject, rand)
# tablename

In [17]:
server='xxxx'
port='5439'
dbase='xxx'
user='xxx'
password='xxx'

pr.connect_to_redshift(dbname = dbase,
                       host = server,
                       port = port,
                       user = user,
                       password = password)

In [18]:
data_requests_log_id="""
SELECT MAX(id)+1 as id_val from stg_data_genie.data_requests_log;"""
id_val=pr.redshift_to_pandas(data_requests_log_id)
id_val=id_val['id_val'][0]
id_val

91

In [19]:
data_request1="""
SELECT {attributes_str}
FROM xxx
LEFT JOIN xxx p
	ON vuab.provider_buyer_name = p.provider_buyer_id
LEFT JOIN xxx ss
	ON vuab.conversion_status_id = ss.survey_status_id
WHERE {filters_str};
""".format(attributes_str=attributes_str,
           filters_str=filters_str)

In [20]:
print(data_request1)


SELECT CASE WHEN vuab.provider_id != 18 THEN vuab.provider_name WHEN vuab.provider_id = 18 THEN p.provider_buyer_name END AS provider, vuab.activity_id AS survey_id, vuab.user_id, vuab.click_event_datetime_pt, CASE WHEN vuab.conversion_status_id IS NULL THEN 'DropOff' ELSE CONCAT(CONCAT(ss.survey_status_id, ' '), ss.survey_status_name) END AS survey_status_name
FROM prd_edw.vw_uab_silver vuab
LEFT JOIN prd_edw.dim_disqo_audience_provider_buyer p
	ON vuab.provider_buyer_name = p.provider_buyer_id
LEFT JOIN prd_edw.dim_survey_status ss
	ON vuab.conversion_status_id = ss.survey_status_id
WHERE provider in ('Think Now') and DATE(vuab.click_event_datetime_pt) >= '2022-02-03 00:00:00' and DATE(vuab.click_event_datetime_pt) <= '2022-02-03 00:00:00' and vuab.conversion_status_id IS NULL;



In [21]:
data_requests_log_insert="""
INSERT INTO stg_data_genie.data_requests_log
(id, DR_type, request_date, analyst_initials, requester_initials, request_subject, 
 salt, create_timestamp, table_name, query)
VALUES (
  {id_val},
  '{DR_type}',
  '{request_date}',
  '{analyst_initials}',
  '{requester_initials}',
  '{request_subject}',
  '{salt}',
  '{create_timestamp}',
  '{table_name}',
  '{query}');
""".format(id_val=id_val,
           DR_type='API2',
           request_date=date,
           analyst_initials='PG',
           requester_initials=initials,
           request_subject=subject,
           salt='',
           create_timestamp=dt.datetime.utcnow(),
           table_name='N/A',
           query='N/A')
pr.exec_commit(data_requests_log_insert) 

In [22]:
start=dt.datetime.now()

data_request=pr.redshift_to_pandas(data_request1)

end=dt.datetime.now()
print('minutes to complete query:', (end-start).seconds//60)

minutes to complete query: 0


#### Review & Export

In [23]:
data_request.head()

,provider,survey_id,user_id,click_event_datetime_pt,survey_status_name
0,Think Now,64000-gQ9qta,23848498,2022-02-03 13:02:01,DropOff
1,Think Now,64000-HzeLJI,24092032,2022-02-03 12:19:30,DropOff
2,Think Now,64000-PrgAxL,22805018,2022-02-03 12:20:28,DropOff
3,Think Now,64000-wdgWVs,23978812,2022-02-03 07:42:27,DropOff
4,Think Now,64000-epr1oM,24119548,2022-02-03 07:44:21,DropOff


In [24]:
export_folder="/Users/peter.gurka/Google Drive/My Drive/Self Serve Reporting/API 2 API Project Data Request/Outputs/"

In [25]:
csv_export_1 = export_folder + """API2_{0}_{1}_{2}_{3}.csv""".format(initials, subject, date, rand)

In [26]:
data_request.to_csv(csv_export_1)

In [27]:
data_requests_log_update="""
UPDATE stg_data_genie.data_requests_log 
SET complete_timestamp='{complete_timestamp}'
WHERE id={id_val};
""".format(complete_timestamp=dt.datetime.utcnow(),
           id_val=id_val)
pr.exec_commit(data_requests_log_update)

In [28]:
pr.close_up_shop()

In [29]:
job_end=dt.datetime.utcnow()
job_runtime=(job_end-job_start).seconds//60
print('minutes to complete job:', job_runtime)

minutes to complete job: 0


#### Post on Slack

In [30]:
slack_message_end = """{0}'s API2 (CSM) request for {1} has completed in {2} minutes"""\
                    .format(initials, subject, job_runtime)
slack_data = {'text': slack_message_end}

In [31]:
response = requests.post(
    webhook_url, data = json.dumps(slack_data),
    headers={'Content-Type': 'application/json'})
if response.status_code != 200:
    raise ValueError(
        'Request to slack returned an error %s, the response is:\n%s'
        % (response.status_code, response.text))